In [7]:
#https://www.tensorflow.org/tutorials/load_data/pandas_dataframe

In [1]:
import os
import tensorflow as tf
import tensorflow.contrib.eager as tfe
# tf.compat.v1.enable_eager_execution()
print(tf.__version__)

import time
import ingestor
import extractor2
import utils
import data
import functools

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

1.14.0
Tesor Flow Version: 1.14.0 Ingest File
Tesor Flow Version: 1.14.0  Utility File
Tesor Flow Version: 1.14.0 Extactor 2


In [2]:
params = ingestor.get_params()

In [3]:
train_data = data.get_train_dataset(params)
parser = data.tristan_parser
train_parsed = train_data.map(functools.partial(parser, params=params))

In [4]:
# for i in train_parsed.take(1):
#     print(i)
# tf.Tensor(376967, shape=(), dtype=int64)

In [7]:
TAKE = 10000

In [8]:
%%time
# set features and labels
features = {'goal': [], 'goal_asl': [], 'thms': [], 'thms_hard_negatives': []}
labels = {'tac_id': []}

# iterate over dataset to extract data into arrays. remove 'take' part to iterate over the entire dataset
for raw_record in train_parsed.take(TAKE):
    fx, lx = raw_record[0], raw_record[1]
    features['goal'].append(fx['goal'])
    features['goal_asl'].append(fx['goal_asl'])
    features['thms'].append(fx['thms'])
    features['thms_hard_negatives'].append(fx['thms_hard_negatives'])
    labels['tac_id'].append(lx['tac_id'])

# instantiate extractor object
ex = extractor2.Extractor(params)

# tokenize goals
temp = ex.tokenize(features['goal'], ex.vocab_table)
#pad all goals to be of length 1000
goal_list = []
for j in range(len(temp)):
        l = len(temp[j])
        h = tf.pad(temp[j], [[0, 1000-l]], constant_values=0)
        goal_list.append(h)
features['goal_ids'] = goal_list

# tokenize hypotheses. this requires more work since there may be more than one hypothesis
length = len(features['goal'])
features['goal_asl_ids'] = []

for i in range(length):
    temp = ex.tokenize(features['goal_asl'][i], ex.vocab_table)
    #pad all hypotheses to be of length 1000
    hypo_list = []
    for j in range(len(temp)):
        l = len(temp[j])
        h = tf.pad(temp[j], [[0, 1000-l]], constant_values=0)
        hypo_list.append(h)
    features['goal_asl_ids'].append(hypo_list)

del features['goal']
del features['goal_asl']
del features['thms']
del features['thms_hard_negatives']

# FEATURES
# 'goal_ids': an array of LENGTH OF DATASET containing arrays which are the tokenized goals of length 1000
# below is an example of how to access the entire array as numpy arrays
# print(features['goal_ids'][0].numpy())
# 'goal_asl_ids': this below is an array of LENGTH OF DATASET containing lists of tokenized hypotheses 
# where each hypothesis is of length 1000. below is an example of how to access the first hypothesis 
# from the list of hypotheses corresponding to the first training example
# print(features['goal_asl_ids'][0][0].numpy())

# LABELS
# 'tac_id': array containing tactic ids. below is an example of how to access the first tactic
# print(labels['tac_id'][0].numpy())


Instructions for updating:
Use `tf.cast` instead.
CPU times: user 57.8 s, sys: 642 ms, total: 58.4 s
Wall time: 1min 3s


In [9]:
tf.shape(features['goal_ids'])

<tf.Tensor: id=1153453, shape=(2,), dtype=int32, numpy=array([10000,  1000], dtype=int32)>

In [10]:
tf.shape(labels['tac_id'])

<tf.Tensor: id=1153456, shape=(1,), dtype=int32, numpy=array([10000], dtype=int32)>

In [11]:
## More succing version of creatig a dataset
## Need to understand how to do batching
def DataframeCreator():
    
    #create an array of features
    features_array = np.zeros((TAKE,1000))
    for i,x in enumerate(features['goal_ids']):
        features_array[i] = x
    
    #create an array of labels
    labels_array = np.zeros((TAKE))
    for i,x in enumerate(labels['tac_id']):
        labels_array[i] = x
        
    
    # put these together in a pandas dataframe
    # perhaps not the most efficient thing but allows for correct types 
    # and quick data analysis
    
    #not necessary to have col names but since we will also be adding hypothesis this may be necessary 
    feature_cols = []
    for i in range(1000):
        feature_cols.append("h" + str(i))
        
    dataset = pd.DataFrame(features_array, columns= feature_cols)
    dataset["label"] = labels_array
    
    return dataset
        
    
    

In [12]:
df = DataframeCreator()

In [20]:
df.head()

,h0,h1,h2,h3,h4,h5,h6,h7,h8,h9,...,h991,h992,h993,h994,h995,h996,h997,h998,h999,label
0,2.0,6.0,9.0,4.0,4.0,4.0,8.0,5.0,12.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34.0
1,2.0,6.0,6.0,9.0,4.0,8.0,5.0,13.0,4.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37.0
2,2.0,6.0,6.0,9.0,4.0,5.0,4.0,5.0,7.0,43.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
3,2.0,6.0,9.0,4.0,4.0,5.0,7.0,7.0,40.0,18.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
4,2.0,6.0,9.0,4.0,4.0,4.0,8.0,5.0,15.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0


In [14]:
def TrainTestSets(dataset):
    '''here features and labels are in one object not sure if that is an issue, 
followed a tutorial buy may have to split?'''

    train, test = train_test_split(dataset, test_size=0.2)
    target_train = train.pop("label")
    Train = tf.data.Dataset.from_tensor_slices((train.values, target_train.values))
    Train2 = Train.shuffle(len(train)).batch(1)
    
    
    target_test = test.pop("label")
    Test = tf.data.Dataset.from_tensor_slices((test.values, target_test.values))
    Test2 = Test.shuffle(len(test)).batch(1)

    return Train2, Test2
    


In [15]:
train_object, test_object = TrainTestSets(df)

In [93]:
train_object

<DatasetV1Adapter shapes: ((?, 1000), (?,)), types: (tf.float64, tf.float64)>

In [67]:
import pytorch

ModuleNotFoundError: No module named 'pytorch'

In [ ]:
#'glorot_uniform'

In [135]:
def get_compiled_model():
    model = tf.keras.Sequential([
    tf.keras.layers.Dense(124, input_shape = (1000,), activation='relu', kernel_initializer = 'glorot_uniform'),
    tf.keras.layers.Dropout(0.8),
    tf.keras.layers.Dense(256, input_shape = (1000,), activation='relu', kernel_initializer = 'glorot_uniform'),
    tf.keras.layers.Dropout(0.8),
    tf.keras.layers.Dense(41, activation='relu', kernel_initializer = 'glorot_uniform'),
    tf.keras.layers.Softmax(-1)
  ])
    
    model.compile(optimizer='Adam',
                loss='categorical_crossentropy',
                metrics=['categorical_accuracy'])
    return model

In [136]:
model = get_compiled_model()

In [ ]:
for i in range(5):
    model.fit(train_object, epochs=4)
    test_loss, test_acc = model.evaluate(test_object, verbose=2)

Epoch 1/4
8000/8000 [==============================] - 37s 5ms/step - loss: 2512.9860 - categorical_accuracy: 0.0000e+00
Epoch 2/4
8000/8000 [==============================] - 37s 5ms/step - loss: 2512.2509 - categorical_accuracy: 0.0615
Epoch 3/4
8000/8000 [==============================] - 37s 5ms/step - loss: 2514.4244 - categorical_accuracy: 0.0325
Epoch 4/4
8000/8000 [==============================] - 37s 5ms/step - loss: 2510.3200 - categorical_accuracy: 5.0000e-04
2000/2000 - 5s - loss: 2502.1167 - categorical_accuracy: 0.0015
Epoch 1/4
8000/8000 [==============================] - 37s 5ms/step - loss: 2508.7736 - categorical_accuracy: 0.0036
Epoch 2/4
8000/8000 [==============================] - 37s 5ms/step - loss: 2509.9572 - categorical_accuracy: 0.0126
Epoch 3/4
8000/8000 [==============================] - 37s 5ms/step - loss: 2510.2582 - categorical_accuracy: 0.0105
Epoch 4/4
8000/8000 [==============================] - 37s 5ms/step - loss: 2508.4540 - categorical_accuracy:

In [121]:
# import matplotlib.pyplot as plt 
# lables = df['label']
# plt.hist(lables, bins=41)

In [143]:
model.predict(row[0])[0]

array([0.02440099, 0.02439384, 0.02439534, 0.02439257, 0.0243917 ,
       0.02439156, 0.02439199, 0.02439313, 0.02439625, 0.02424204,
       0.02439204, 0.0243916 , 0.02439112, 0.02439501, 0.02439119,
       0.02439912, 0.02439206, 0.02439129, 0.02439151, 0.02439477,
       0.02439339, 0.02442904, 0.0243953 , 0.02439228, 0.02439201,
       0.02439235, 0.0243923 , 0.02439236, 0.02439203, 0.02439222,
       0.02439131, 0.02439211, 0.02439087, 0.02439093, 0.02439181,
       0.02439218, 0.02439163, 0.02439233, 0.02439227, 0.02439114,
       0.02440105], dtype=float32)

In [123]:
for row in test_object.take(1):
    print(row[1])

tf.Tensor([3.], shape=(1,), dtype=float64)


In [39]:
# np.array(row[0])

In [87]:
print(model.layers)

[<tensorflow.python.keras.layers.core.Dense object at 0x7f97b8052128>, <tensorflow.python.keras.layers.core.Dense object at 0x7f979b5ef208>, <tensorflow.python.keras.layers.core.Dense object at 0x7f979b5ef8d0>]


In [112]:
print(model.summary())

Model: "sequential_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_78 (Dense)             (None, 124)               124124    
_________________________________________________________________
dense_79 (Dense)             (None, 64)                8000      
_________________________________________________________________
softmax_1 (Softmax)          (None, 64)                0         
Total params: 132,124
Trainable params: 132,124
Non-trainable params: 0
_________________________________________________________________
None


In [113]:
for layer in model.layers:
    print("Input shape: "+str(layer.input_shape)+". Output shape: "+str(layer.output_shape))

Input shape: (None, 1000). Output shape: (None, 124)
Input shape: (None, 124). Output shape: (None, 64)
Input shape: (None, 64). Output shape: (None, 64)


In [ ]:
EPOCHS=10
